### Data Collection II
Making it easier to access multiple series of the M3 survey from Census Bureau

In [ ]:
import pandas as pd
import requests
import json
import os 
from bs4 import BeautifulSoup as bs 
import re
from collections import OrderedDict as od
from pypdf import PdfReader 

In [ ]:
os.chdir("C:/Users/marvi/OneDrive/Documents/GitHub/Scraping-and-APIs")
os.getcwd()

#### Census Bureau
[M3 categories](https://www2.census.gov/programs-surveys/m3/technical-documentation/code-lists/sichist.pdf)

In [ ]:
census = requests.get("https://api.census.gov/data/timeseries/eits/m3.json?get=cell_value,data_type_code,time_slot_id,seasonally_adj,category_code,error_data&for=US&time=from+1992-01+to+2024-03&key=a9c6bd12867708842174babc90f20cfaa20255a7")
census = census.json()
cbs = pd.DataFrame(census, columns = census[0])
cbs = cbs[cbs["data_type_code"] == 'VS'].reset_index(drop = True)

In [ ]:
#Grabs and sorts unique values for the category code
b = cbs["category_code"].tolist()
cat_abv = list(od.fromkeys(b))
cat_abv = sorted(cat_abv)

In [ ]:
reader = PdfReader('m3_codes.pdf')
print(len(reader.pages))
text = ""

for i in range(0, len(reader.pages)):
    text += reader.pages[i].extract_text()
text = str(text)


In [ ]:
for i in range(0,len(text)):
    if (text[i] == " "):
        print("blank")
